In [1]:
import pyspark
from pyspark.sql import SparkSession,DataFrame
import requests
import json 
from io import BytesIO
import pandas as pd
import os
import sys
import utils as utils

In [8]:
path = "https://dataestur.azure-api.net/API-SEGITTUR-v1/TURISMO_INTERNO_PROV_CCAA_DL"
filter = "CCAA%20origen=Todos&Provincia%20origen=Todos&CCAA%20destino=Todos&Provincia%20destino=Todos"
# https://dataestur.azure-api.net/API-SEGITTUR-v1/TURISMO_INTERNO_PROV_CCAA_DL?CCAA%20origen=Todos&Provincia%20origen=Todos&CCAA%20destino=Todos&Provincia%20destino=Todos
spark = utils.create_context()
db_name = "landing"
table_name = "turismo_Provincia"


if utils.check_table_exists(spark,db_name,table_name):


    df = utils.get_api_endpoint_data(spark,path,filter)
    utils.overwrite_iceberg_table(spark,df,db_name,table_name)
    
df = utils.get_api_endpoint_data(spark,path,filter)

utils.overwrite_iceberg_table(spark,df,db_name,table_name)
spark.stop()

c:\Users\joaqu\.virtualenvs\TravelMind--WcSvJCM\Lib\site-packages\pyspark\sql\catalog.py:748: FutureWarning: `dbName` has been deprecated since Spark 3.4 and might be removed in a future version. Use tableExists(`dbName.tableName`) instead.
  warnings.warn(
